# Test Word2vec

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim, logging
import gensim.models.keyedvectors as word2vec
import matplotlib.pyplot as plt

%matplotlib inline

# Data

In [2]:
# Importing prepared dataset
import pickle
input = open('my_corpus.pql', 'rb')
obj = pickle.load(input)
input.close()

corpus = obj['corpus']
words_list = obj['words_list']
all_words = obj['all_words']
print(corpus[1])

народитися дивитися категорія народитися грехема макферсон англійський співак джулія дрейфус американський актриса майя чибурданідзе грузинський шахістка уейн грецкий канадський хокеїст лютий геннадій куцак заслужений лютий вінс нейл американський співак лютий енді тейлор англійський музикант лютий коротаєв андрій віталійович історик філософ соціолог орієнталіст лютий джон американський музикант олена олексіївна яковлев російський кіноактриса лотар маттеус німецький футболіст олександр цекати співак василь радянський футболіст едді мерфі актор олександр заваров футболіст травень джей астон співак травень джордж клуня американський актор травень кемпбелл музикант травень денніс родмен американський баскетболіст травень англійський кіноактор травень томпсон актриса майкл фокс актор джордж англійський співак елісон співачка наталія марія петрівна фарина співачка сопрано педагог заслужений артистка


## Splitting

In [3]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
w1, w2 = train_test_split(words_list, test_size=0.5, random_state=10)

# W2V

In [4]:
model = word2vec.KeyedVectors.load_word2vec_format('ubercorpus.lowercased.lemmatized.word2vec.300d.txt', binary=False)

In [5]:
words = list(model.key_to_index)

In [6]:
def txt_embed(words, corpus):
    x_sent_embed = []
    count_words, count_non_words = 0, 0  
    
    # recover the embedding of each sentence with the average of the vector that composes it
    # sent - sentence
    for sent in corpus:
        # average embedding of all words in a sentence
        sent_embed = []
        for word in sent:
            try:
                # if word is present in the dictionary - add its vector representation
                count_words += 1
                sent_embed.append(model[word])
            except KeyError:
                # if word is not in the dictionary - add a zero vector
                count_non_words += 1
                sent_embed.append([0] * 300)
        
        # add a sentence vector to the list
        x_sent_embed.append(np.mean(sent_embed, axis=0).tolist())
                    
    print(count_non_words, "out of", count_words, "words were not found in the vocabulary.")
    
    return x_sent_embed

In [7]:
x1 = txt_embed(words, w1)
x2 = txt_embed(words, w2)

44333 out of 367958 words were not found in the vocabulary.
47963 out of 372782 words were not found in the vocabulary.


In [8]:
embed1 = []
for i,v in enumerate(x1): 
    if np.any(np.isnan(v)) != True:
        embed1.append(np.array(v))
    else:
        
        embed1.append(np.array([0] * 300))
        print(i)
em1 = np.vstack(embed1)

In [9]:
embed2 = []
for i,v in enumerate(x2): 
    if np.any(np.isnan(v)) != True:
        embed2.append(np.array(v))
    else:
        embed2.append(np.array([0] * 300))
        print(i)
em2 = np.vstack(embed2)

## Metrics

In [10]:
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import v_measure_score

In [11]:
def metrics(model1, model2):
    rand = adjusted_rand_score(model1,model2).round(3)
    homo = homogeneity_score(model1,model2).round(3)
    vm = v_measure_score(model1,model2).round(3)
    print(rand, homo, vm)

# K-means

In [12]:
# Number of Clusters
k = 9

In [13]:
# K-means clustering
from sklearn.cluster import KMeans
km1 = KMeans(n_clusters = k).fit(em1)
km2 = KMeans(n_clusters = k).fit(em2)

# Comparing Clustering Algorithms
metrics(km1.labels_.tolist(),km2.labels_.tolist())

-0.0 0.008 0.008


## W2V AgglomerativeClustering

In [16]:
from sklearn.cluster import AgglomerativeClustering
hc1 = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'ward').fit_predict(em1)
hc2 = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'ward').fit_predict(em2)

# Comparing Clustering Algorithms
metrics(hc1,hc2)

0.0 0.009 0.009


## MeanShift

In [21]:
from sklearn.cluster import MeanShift
mshift1 = MeanShift(bandwidth=11).fit(em1)
mshift2 = MeanShift(bandwidth=11).fit(em2)

# Comparing Clustering Algorithms
metrics(mshift1.labels_,mshift2.labels_)

-0.006 0.003 0.001


In [22]:
len(set(mshift1.labels_))

6

## HDBSCAN

In [19]:
import hdbscan
hdb1 = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=7).fit(em1)
hdb2 = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=7).fit(em2)

# Comparing Clustering Algorithms
metrics(hdb1.labels_,hdb2.labels_)

0.007 0.007 0.009


In [20]:
len(set(hdb2.labels_))

6